In [ ]:
# 1️⃣ Install cloudflared
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

# 2️⃣ Run the FastAPI embedding server
!pip install sentence-transformers fastapi uvicorn

--2025-08-21 11:33:34--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.8.0/cloudflared-linux-amd64 [following]
--2025-08-21 11:33:34--  https://github.com/cloudflare/cloudflared/releases/download/2025.8.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/f51b16ea-9165-4877-a4ed-498a2db75169?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-21T12%3A22%3A41Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-21T1

In [ ]:
# Create embedding server script
with open("embed_server.py", "w") as f:
    f.write("""
from fastapi import FastAPI
from pydantic import BaseModel
from sentence_transformers import SentenceTransformer
from google.colab import runtime
import threading, time

app = FastAPI()

print("📥 Loading embedding model...")
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
print("✅ Model loaded.")

class EmbedRequest(BaseModel):
    texts: list[str]

@app.post("/embed")
def embed(req: EmbedRequest):
    embeddings = model.encode(req.texts).tolist()
    return {"embeddings": embeddings}

@app.post("/kill")
def kill():
  def shutdown():
    time.sleep(2)
    runtime.unassign()

  threading.Thread(target=shutdown).start()
  return {"status": "Colab runtime will be unassigned in 2s"}

""")



In [ ]:
import subprocess, time, requests

# Start FastAPI server
server_proc = subprocess.Popen(
    ["uvicorn", "embed_server:app", "--host", "0.0.0.0", "--port", "8000"]
)

# Wait until it's live
time.sleep(5) # Add a 5-second delay before checking
for i in range(60):  # Increase the timeout to 60 seconds
    try:
        r = requests.get("http://localhost:8000/docs")
        if r.status_code == 200:
            print("✅ FastAPI is live!")
            break
    except requests.exceptions.ConnectionError:
        pass
    time.sleep(1)
else:
    raise RuntimeError("❌ FastAPI did not start in time")

# Start cloudflared tunnel
cloudflared_proc = subprocess.Popen(
    ["./cloudflared", "tunnel", "--url", "http://localhost:8000", "--no-autoupdate"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Print the public URL when ready
import re
for line in cloudflared_proc.stdout:
    print(line.strip())
    m = re.search(r"(https://.*\.trycloudflare\.com)", line)
    if m:
        print(f"\n🌍 Public URL: {m.group(1)}/embed")
        break

✅ FastAPI is live!
2025-08-21T11:34:15Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-08-21T11:34:15Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-08-21T11:34:19Z INF +--------------------------------------------------------------------------------------------+
2025-08-21T11:34:19Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-08-21T11:34:19Z INF |  https://healthy-sister-instructors

In [ ]:

# # Get the public URL
# import re, time
# public_url = None
# for _ in range(30):
#     line = cloudflared_proc.stdout.readline()
#     print(line.strip())
#     match = re.search(r"(https://.*\.trycloudflare\.com)", line)
#     if match:
#         public_url = match.group(1)
#         break
#     time.sleep(1)

# if public_url:
#     print(f"\n✅ Public URL for embedding server:\n{public_url}")
# else:
#     print("❌ Could not get public URL")